In [1]:
import scrapy
import re
import pandas as pd
from scrapy.crawler import CrawlerProcess

In [2]:
class rt_spider (scrapy.Spider):
    name="rt_sider"
    

    
    #create start requests
    def start_requests (self):
        url = "https://editorial.rottentomatoes.com/guide/essential-movies-to-watch-now/"
        pages=[1,2]
        for page in pages:
            new_url = url + str(page)+ '/'
            yield scrapy.Request(url = new_url, callback = self.page_parse)
        
    def page_parse(self, response):
        print(response.url)
        for movie in response.css('div[id*= "row-index-"] > div.col-sm-18.col-full-xs.countdown-item-content > div.row.countdown-item-title-bar > div.col-sm-20.col-full-xs > div > div > h2 > a::attr(href)').extract():     
            yield response.follow(url = movie, callback = self.movie_page_parse)
            
    def movie_page_parse(self, response):
        
        
        
        def retrive (response , key):
            '''
            retrive data based on key search 
            '''
            for li in response.css('ul.content-meta.info > li'):
                if li.css('div.meta-label.subtle::text').extract()[0] == key:      
                    list_of_elem = [e.strip().replace('\n',"")  for e in li.css('div.meta-value ::text').extract()]
                    if "," in list_of_elem:
                        value = [e for e in list_of_elem if e not in ['', ","]][0] + ', ' + [e for e in list_of_elem if e not in ['', ","]][1]
                    else:
                        value = [e for e in list_of_elem if e not in ['', ","]][0]
                    return value
            
        movie_url = response.url
        movie_name= response.css('#topSection > div.thumbnail-scoreboard-wrap > score-board > h1::text').extract()[0]
        
        audience_score = response.css('score-board ::attr(audiencescore)').extract()[0]
        audience_rating = response.css('#topSection > div.thumbnail-scoreboard-wrap > score-board > a.scoreboard__link.scoreboard__link--audience::text').extract()[0].strip()[:-9].replace(',','')
        rt_score = response.css('score-board ::attr(tomatometerscore)').extract()[0]
        rt_reviwers=response.css('#topSection > div.thumbnail-scoreboard-wrap > score-board > a.scoreboard__link.scoreboard__link--tomatometer::text').extract()[0].strip()[:-8].replace(',','')
        

        genre= response.css('#topSection > div.thumbnail-scoreboard-wrap > score-board > p::text').extract()[0].split(',')[1].strip()
        release = response.css('#topSection > div.thumbnail-scoreboard-wrap > score-board > p::text').extract()[0].split(',')[0].strip()
        duration = response.css('#topSection > div.thumbnail-scoreboard-wrap > score-board > p::text').extract()[0].split(',')[2].strip()
        
        #convert duration from text to minutes
        time= duration.split()
        x=1
        if len(time) == x :
            time_min = int(re.findall(r'\d+', time[0])[0])*60
        else:
            time_min=int(re.findall(r'\d+', time[0])[0])*60 + int (re.findall(r'\d+', time[1])[0])

        
        rating= retrive(response, 'Rating:')
        language= retrive(response, 'Original Language:')
        director= retrive(response, 'Director:')
        producer= retrive(response, 'Producer:')
        writer= retrive(response, 'Writer:')



        top_300.append({
                         'name': movie_name,
                        'critics_score' : int(rt_score),
                        'critics_review_numbers' : int(rt_reviwers),
                        'audience_score' : int(audience_score),
                        'audience_reviews' :(audience_rating),
                        'year' : int(release),
                        'genre' : genre,
                        'duration' : duration,
                        'time_in_minutes' : time_min,
                        'movie_page' : movie_url,
                        'rating' : rating,
                        'language': language,
                        'director': director,
                        'producer':producer,
                        'writer': writer
                        })
    



           

In [ ]:
top_300=[]
process = CrawlerProcess()
process.crawl(rt_spider)
output = process.start()

In [5]:
len(top_300)

300

In [6]:
df=pd.DataFrame(top_300, columns=['name','critics_score','critics_review_numbers','audience_score', 'audience_reviews',
                                  'year','genre', 'duration','time_in_minutes', 'movie_page','rating', 'language', 'director', 
                                  'producer', 'writer'])

In [8]:
df.sample(10)

,name,critics_score,critics_review_numbers,audience_score,audience_reviews,year,genre,duration,time_in_minutes,movie_page,rating,language,director,producer,writer
97,Say Anything...,98,48,85,100000,1989,Romance,1h 40m,100,https://www.rottentomatoes.com/m/say_anything,PG-13,English,Cameron Crowe,Polly Platt,Cameron Crowe
26,Portrait of a Lady on Fire,97,332,92,250+ Verifie,2019,Romance/History,2h 1m,121,https://www.rottentomatoes.com/m/portrait_of_a...,R (Some Nudity and Sexuality),French (France),Céline Sciamma,"Bénédicte Couvreur, Véronique Cayla",Céline Sciamma
176,King Kong,96,113,86,50000,1933,Adventure/Fantasy,1h 40m,100,https://www.rottentomatoes.com/m/1011615-king_...,None,English,"Merian C. Cooper, Ernest B. Schoedsack","Merian C. Cooper, Ernest B. Schoedsack","James Ashmore Creelman, Ruth Rose"
62,This Is Spinal Tap,95,66,92,100000,1984,Comedy/Music,1h 22m,82,https://www.rottentomatoes.com/m/this_is_spina...,R,English,Rob Reiner,Karen Murphy,"Christopher Guest, Michael McKean"
288,All About My Mother,98,95,93,50000,1999,Drama/Comedy,1h 39m,99,https://www.rottentomatoes.com/m/all_about_my_...,R (Sexuality|Language|Some Drug Content|Strong...,Spanish (Spain),Pedro Almodóvar,Agustín Almodóvar,Pedro Almodóvar
152,Nashville,91,75,84,10000,1975,Comedy/Drama,2h 37m,157,https://www.rottentomatoes.com/m/nashville,R,English,Robert Altman,Robert Altman,Joan Tewkesbury
279,Blade Runner,89,126,91,250000,1982,Sci-fi/Mystery & thriller,2h 2m,122,https://www.rottentomatoes.com/m/blade_runner,R,English,Ridley Scott,"Michael Deeley, Charles de Lauzirika","Hampton Fancher, David Webb Peoples"
59,Tokyo Story,100,50,93,10000,1953,Drama,2h 14m,134,https://www.rottentomatoes.com/m/tokyo_story,None,Japanese,Yasujirô Ozu,Takeshi Yamamoto,"Kôgo Noda, Yasujirô Ozu"
203,Goldfinger,99,72,89,100000,1964,Action,1h 52m,112,https://www.rottentomatoes.com/m/goldfinger,PG,English,Guy Hamilton,"Albert R. Broccoli, Harry Saltzman","Ian Fleming, Richard Maibaum"
298,The Adventures of Robin Hood,100,50,89,25000,1938,Adventure/Action,1h 42m,102,https://www.rottentomatoes.com/m/1000355-adven...,PG,English,"Michael Curtiz, William Keighley",Hal B. Wallis,"Norman Reilly Raine, Seton I. Miller"
